In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 2016 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2016 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgWipwpAICsAAEA7SP8AAAAK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2016","wgTitle":"List of Billboard Hot 100 top-ten singles in 2016","wgCurRevisionId":909740527,"wgRevisionId":909740527,"wgArticleId":48877391,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [4]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [11]:
swiftcount = 0 # Keeps track of where repeat information from cells with rowspans greater than 1 should be inserted.
drakecontingency = 0 # Used to keep track of how many times repeat information should be inserted.
swiftcount2 = 0 # Used in the rare case that two rowspan situations overlap.
drakecontingency2 = 0

# The following code mines the first table found on the Wikipedia page and extracts all table elements other than the annotations column.

In [13]:
Titles = []
try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tds = my_table.findAll('td')

for td in tds:
        try:
            if td.attrs['rowspan']:
                if drakecontingency == 0 and int(td.attrs['rowspan']) > 1:
                    taylorswift = td.get_text()[:-1]
                    swiftcount = 8
                    drakecontingency = int(td.attrs['rowspan']) - 1
                elif int(td.attrs['rowspan']) > 1:
                    taylorswift2 = td.get_text()[:-1]
                    swiftcount2 = 7
                    drakecontingency2 = int(td.attrs['rowspan']) - 1
        except KeyError:
            pass
        if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
            if swiftcount > 0:
                swiftcount = swiftcount-1
            if swiftcount == 1:
                drakecontingency = drakecontingency-1
                print(taylorswift + ' appended!')
                Titles.append(taylorswift)
                if drakecontingency == 0:
                    taylorswift = ''
                    swiftcount = 0
                else:
                    if drakecontingency2 == 0: # If there's an extra element being inserted before the next time this runs, insert it one sooner next time.
                        swiftcount = 6
                    else:
                        swiftcount = 5
            if swiftcount2 > 0:
                swiftcount2 = swiftcount2-1
            if swiftcount2 == 1:
                drakecontingency2 = drakecontingency2-1
                print(taylorswift2 + ' appended!')
                Titles.append(taylorswift2)
                if drakecontingency2 == 0:
                    taylorswift2 = ''
                    swiftcount2 = 0
                else:
                    if drakecontingency == 0:
                        swiftcount2 = 6
                    else:
                        swiftcount2 = 5
            Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

December 5 appended!
February 13 appended!
February 13 appended!
February 20 appended!
February 20 appended!
March 19 appended!
May 14 appended!
May 14 appended!
May 21 appended!
May 28 appended!
July 2 appended!
October 29 appended!


In [18]:
print(Titles)

['November 14', '"Sorry" (#2) ◁', 'Justin Bieber', '1', 'January 23', '21', 'December 5', '"Love Yourself"[E] (#1) ◁', 'Justin Bieber', '1', 'February 13', '24', 'December 5', '"Here"', 'Alessia Cara', '5', 'February 6', '11', 'December 19', '"Same Old Love"', 'Selena Gomez', '5', 'January 30', '9', 'January 16', '"Stressed Out" (#5)', 'Twenty One Pilots', '2', 'February 27', '14', 'February 6', '"Roses"', 'The Chainsmokers featuring Rozes', '6', 'February 13', '6', 'February 13', '"My House"', 'Flo Rida', '4', 'March 19', '11', 'February 13', '"Hands to Myself"', 'Selena Gomez', '7', 'February 13', '1', 'February 13', '"Work" (#4) ◁', 'Rihanna featuring Drake', '1', 'March 5', '18', 'February 20', '"Pillowtalk" ◁', 'Zayn', '1', 'February 20', '16', 'February 20', '"Summer Sixteen" ◁', 'Drake', '6', 'February 20', '1', 'February 20', '"Me, Myself & I"', 'G-Eazy and Bebe Rexha', '7', 'March 12', '12', 'March 5', '"Cake by the Ocean"[C]', 'DNCE', '9', 'March 12', '7', 'March 19', '"Piece

In [19]:
nptitles = np.asarray(Titles).copy()

In [20]:
len(nptitles)

276

In [21]:
numsongs = int(np.prod(nptitles.shape)/6) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,6) # Reshape the array based on the number of songs and the number of columns.

In [22]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Top ten entry date','Title','Artist','Peak','Peak date','Number of weeks in top ten']
songdata['Year'] = str(year)
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
for j in songdata['Top ten entry date']:
    if '[' in j:
        j = j[:j.rfind('[')-1]

if year == now.year:
    cleanweeks = []
    for j in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
        try:                                         # problem in the current year.
            cleanweeks.append(int(j[:j.rindex('*')]))
        except ValueError:
            cleanweeks.append(int(j))

    songdata['Number of weeks in top ten'] = cleanweeks

In [23]:
songdata

,Top ten entry date,Title,Artist,Peak,Peak date,Number of weeks in top ten,Year
0,November 14,Sorry,Justin Bieber,1,January 23,21,2016
1,December 5,Love Yourself,Justin Bieber,1,February 13,24,2016
2,December 5,Here,Alessia Cara,5,February 6,11,2016
3,December 19,Same Old Love,Selena Gomez,5,January 30,9,2016
4,January 16,Stressed Out,Twenty One Pilots,2,February 27,14,2016
5,February 6,Roses,The Chainsmokers featuring Rozes,6,February 13,6,2016
6,February 13,My House,Flo Rida,4,March 19,11,2016
7,February 13,Hands to Myself,Selena Gomez,7,February 13,1,2016
8,February 13,Work,Rihanna featuring Drake,1,March 5,18,2016
9,February 20,Pillowtalk,Zayn,1,February 20,16,2016


In [173]:
songdata.to_csv(path_or_buf='data/' + str(year) + 'top10songs.csv',index=False) # Export our sample data table.